### 1. Import thư viện cần thiết

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

### 2. Khởi tạo Spark

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("IE212_Lab2_Bai4") \
    .getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
25/11/25 21:56:42 WARN Utils: Your hostname, ELUCIDATOR resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/25 21:56:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 21:56:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/25 21:56:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/25 21:56:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/11/25 21:56:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


### 3. Load dữ liệu

In [3]:
movies_path = "data/movies.txt"
ratings1_path = "data/ratings_1.txt"
ratings2_path = "data/ratings_2.txt"
users_path = "data/users.txt"

In [4]:
movies_rdd = sc.textFile(movies_path)

In [5]:
ratings_rdd = sc.textFile(f"{ratings1_path},{ratings2_path}")

In [6]:
users_rdd = sc.textFile(users_path)

### 4. Giải quyết bài toán

In [7]:
# 2. Xử lý Users: Lấy UserID và phân loại Age Group ngay từ đầu
# Quy tắc phân nhóm: 0-18, 18-35, 35-50, 50+
def get_age_group_index(age):
    age = int(age)
    if age < 18:
        return 0  # Nhóm 0-18
    elif age < 35:
        return 1  # Nhóm 18-35
    elif age < 50:
        return 2  # Nhóm 35-50
    else:
        return 3  # Nhóm 50+

def parse_user_age(line):
    # Schema users: UserID, Gender, Age, ...
    parts = line.split(",") 
    user_id = parts[0]
    age = parts[2]
    group_idx = get_age_group_index(age)
    return (user_id, group_idx)

user_age_rdd = users_rdd.map(parse_user_age)

In [8]:
# 3. Xử lý Ratings: Chuẩn bị UserID để Join
def parse_rating_for_join(line):
    parts = line.split(",")
    user_id = parts[0]
    movie_id = parts[1]
    rating = float(parts[2])
    return (user_id, (movie_id, rating))

ratings_user_key = ratings_rdd.map(parse_rating_for_join)

In [9]:
# 4. Join Users và Ratings
# Kết quả: (UserID, (AgeGroupIndex, (MovieID, Rating)))
joined_age_data = user_age_rdd.join(ratings_user_key)

In [10]:
# 5. Chuyển đổi sang Vector thống kê 4 nhóm
# Input: (UserID, (GroupIdx, (MovieID, Rating)))
# Output: (MovieID, (Sum0, Cnt0, Sum1, Cnt1, Sum2, Cnt2, Sum3, Cnt3))
def map_to_age_vector(record):
    data = record[1]
    group_idx = data[0]
    movie_data = data[1]
    movie_id = movie_data[0]
    rating = movie_data[1]
    
    # Tạo vector chứa 8 phần tử (4 nhóm x 2 chỉ số sum/count)
    # Mặc định là 0 hết
    vector = [0.0, 0] * 4 
    
    # Điền dữ liệu vào đúng vị trí của nhóm tuổi đó
    # Ví dụ: Nhóm 1 (18-35) sẽ điền vào index 2 và 3
    vector[group_idx * 2] = rating      # Sum
    vector[group_idx * 2 + 1] = 1       # Count
    
    return (movie_id, tuple(vector))

In [11]:
# Reduce: Cộng dồn vector
# Python cho phép cộng tuple dễ dàng bằng lambda
movie_age_stats = joined_age_data.map(map_to_age_vector).reduceByKey(lambda a, b: tuple(x + y for x, y in zip(a, b)))

In [12]:
# 6. Join với tên phim (Movies)
def parse_movie_title(line):
    parts = line.split(",")
    return (parts[0], parts[1])

movies_info = movies_rdd.map(parse_movie_title)

In [13]:
# Kết quả: (MovieID, ((Vector8PhanTu), Title))
final_data = movie_age_stats.join(movies_info)

In [14]:
# 7. Format kết quả cuối cùng
def format_age_result(record):
    stats = record[1][0]
    title = record[1][1]
    
    labels = ["0-18", "18-35", "35-50", "50+"]
    output_parts = []
    
    for i in range(4):
        total_score = stats[i * 2]
        count = stats[i * 2 + 1]
        
        if count > 0:
            avg = total_score / count
            output_parts.append(f"{labels[i]}: {avg:.2f}")
        else:
            output_parts.append(f"{labels[i]}: NA")
            
    # Ghép chuỗi lại: [0-18: NA, 18-35: 3.44, ...]
    formatted_stats = ", ".join(output_parts)
    return f"{title} [{formatted_stats}]"

In [15]:
# 8. Hiển thị
results = final_data.map(format_age_result).collect()

for line in results:
    print(line)

The Lord of the Rings: The Fellowship of the Ring (2001) [0-18: NA, 18-35: 4.00, 35-50: 3.75, 50+: 4.25]
Sunset Boulevard (1950) [0-18: NA, 18-35: 4.17, 35-50: 4.50, 50+: 4.50]
Mad Max: Fury Road (2015) [0-18: NA, 18-35: 3.36, 35-50: 3.64, 50+: NA]
No Country for Old Men (2007) [0-18: NA, 18-35: 3.79, 35-50: 3.88, 50+: 4.17]
Fight Club (1999) [0-18: NA, 18-35: 3.50, 35-50: 3.50, 50+: 3.50]
The Godfather: Part II (1974) [0-18: NA, 18-35: 3.78, 35-50: 4.25, 50+: NA]
The Social Network (2010) [0-18: NA, 18-35: 4.00, 35-50: 3.67, 50+: NA]
Lawrence of Arabia (1962) [0-18: NA, 18-35: 3.60, 35-50: 3.32, 50+: 3.75]
The Terminator (1984) [0-18: NA, 18-35: 4.17, 35-50: 4.06, 50+: 3.83]
Gladiator (2000) [0-18: NA, 18-35: 3.43, 35-50: 3.81, 50+: 3.50]
The Silence of the Lambs (1991) [0-18: NA, 18-35: 3.00, 35-50: 3.00, 50+: 4.00]
The Lord of the Rings: The Return of the King (2003) [0-18: NA, 18-35: 3.83, 35-50: 3.86, 50+: 3.50]
E.T. the Extra-Terrestrial (1982) [0-18: NA, 18-35: 3.56, 35-50: 3.83